In [7]:
library(rvest)
library(tidyverse)

In [38]:
url <- 'https://www.statsf1.com/en/circuits.aspx'

In [43]:
s <- session('https://www.statsf1.com/en/default.aspx')

In [48]:
ses <- 'https://www.statsf1.com/en/circuits.aspx' %>% session()

In [45]:
s <- session_jump_to(s, link)

In [51]:
xml_content <- read_xml(url, encoding = "UTF-8")

ERROR: Error in read_xml(url, encoding = "UTF-8"): could not find function "read_xml"


In [42]:
page = read_html(s)

ERROR: [1m[33mError[39m in `read_html()`:[22m
[33m![39m Page doesn't appear to be html.


In [71]:
circuits_df <- page %>% html_node('.sortable') %>% html_table(header = TRUE)

In [72]:
circuits_df <- data.frame(circuits_df)

In [73]:
names(circuits_df)

[1] "X78.Circuits" "Var.2"        "Countries"    "Countries.1"  "Begin.in"    
[6] "Nb"

In [74]:
circuits_df <- circuits_df %>% select('X78.Circuits', 'Var.2','Countries.1')

In [84]:
circuits_df$X78.Circuits <- stri_trans_general(circuits_df$X78.Circuits, "Latin-ASCII")

In [92]:
circuits_df <- circuits_df[-79, ]

In [151]:
track_name <- 'Silverstone'

In [152]:
track <- sprintf('https://www.statsf1.com/en/circuit-%s.aspx', track_name)

In [153]:
page <- read_html(track)

In [154]:
page

{html_document}
<html id="ctl00_html" lang="en">
[1] <head>\n<!-- Google Analytics --><!-- Global site tag (gtag.js) - Google  ...
[2] <body>\r\n  <form name="aspnetForm" method="post" action="/en/circuit-Sil ...

In [155]:
link <- page %>% html_node("#ctl00_CPH_Main_HL_Google") %>% html_attr('href')

In [12]:
library(urltools)

In [157]:
parsed <- url_parse(link)

In [158]:
parsed

scheme,domain,port,path,parameter,fragment
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
https,"52.07159,-1.01615,2778m",NA,data=!3m1!1e3,NA,NA


In [159]:
lats <- strsplit(parsed$domain, ',') %>% unlist

In [160]:
lats

[1] "52.07159" "-1.01615" "2778m"

In [6]:
get_lat <- function(track_name){
    Sys.sleep(1)
    url <- sprintf('https://www.statsf1.com/en/circuit-%s.aspx', track_name)
    page <- read_html(url)
    link <- page %>% html_node("#ctl00_CPH_Main_HL_Google") %>% html_attr('href')
    parsed <- url_parse(link)
    cords <- strsplit(parsed$domain, ',') %>% unlist
    
    return(as.numeric(cords[1]))
}

In [7]:
get_long <- function(track_name){
    Sys.sleep(1)
    url <- sprintf('https://www.statsf1.com/en/circuit-%s.aspx', track_name)
    page <- read_html(url)
    link <- page %>% html_node("#ctl00_CPH_Main_HL_Google") %>% html_attr('href')
    parsed <- url_parse(link)
    cords <- strsplit(parsed$domain, ',') %>% unlist
    
    return(as.numeric(cords[2]))
}

In [161]:
get_long('Silverstone')

[1] -1.01615

In [170]:
circuits_df$X78.Circuits <- gsub(" ", "-",circuits_df$X78.Circuits)

In [106]:
safer_get_lat <- possibly(get_lat, otherwise = NA, quiet = FALSE)
safer_get_long <- possibly(get_long, otherwise = NA, quiet = FALSE)

In [174]:
circuits_df <- circuits_df %>% mutate('lat' = sapply(circuits_df$X78.Circuits, safer_get_lat))

In [175]:
circuits_df <- circuits_df %>% mutate('long' = sapply(circuits_df$X78.Circuits, safer_get_long))

In [178]:
write_csv(circuits_df, 'race_locations.csv')

In [181]:
circuits_df

,X78.Circuits,Var.2,Countries.1,lat,long
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Adelaide,Adelaide Street Circuit,Australia,-34.92700,138.6172609
2,Aida,Okayama International Circuit,Japan,34.91529,134.2207281
3,Ain-Diab,,Morocco,33.58053,-7.6890900
4,Aintree,Aintree Motor Racing Circuit,United Kingdom,53.47590,-2.9413726
5,Anderstorp,Scandinavian Raceway,Sweden,57.26391,13.6030116
6,Austin,Circuit of the Americas,USA,30.13364,-97.6345729
7,Avus,Automobil-Verkehrs- und Übungs-Straße,Germany,52.48658,13.2654900
8,Bakou,Baku Street Circuit,Azerbaijan,40.37126,49.8476935
9,Barcelone,Circuit de Catalunya,Spain,41.56828,2.2582064


In [182]:
url <- 'https://gpracingstats.com/seasons/2022-world-championship/'

In [183]:
page <- read_html(url)

In [191]:
pos <- page %>% html_node('.summary') %>% html_table(header = TRUE)

In [192]:
pos$Circuit <- gsub(' ', '-', pos$Circuit)

In [196]:
pos <- pos %>% mutate('lat' = sapply(pos$Circuit, safer_get_lat), 
                      'long' = sapply(pos$Circuit, safer_get_long))

In [204]:
pos$Circuit[pos$Circuit == 'Jeddah'] = 'Djeddah'
pos$Circuit[pos$Circuit == 'Florida'] = 'Miami'
pos$Circuit[pos$Circuit == 'Marina-Bay'] = 'Singapour'
pos$Circuit[pos$Circuit == 'Monte-Carlo'] = 'Monaco'
pos$Circuit[pos$Circuit == 'Catalunya'] = 'Barcelone'
pos$Circuit[pos$Circuit == 'Baku'] = 'Bakou'

In [206]:
pos <- pos %>% mutate('lat' = sapply(pos$Circuit, safer_get_lat), 
                      'long' = sapply(pos$Circuit, safer_get_long))

In [207]:
pos

#,Grand Prix,Circuit,Date,lat,long
<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Bahrain,Sakhir,20 March,26.030390,50.513543
2,Saudi Arabia,Djeddah,27 March,21.638094,39.099443
3,Australia,Melbourne,10 April,-37.849140,144.969873
4,Emilia Romagna,Imola,24 April,44.339858,11.713329
5,Miami,Miami,08 May,25.957740,-80.238678
6,Spain,Barcelone,22 May,41.568277,2.258206
7,Monaco,Monaco,29 May,43.738095,7.426050
8,Azerbaijan,Bakou,12 June,40.371259,49.847693
9,Canada,Montreal,19 June,45.505890,-73.524110


In [209]:
write_csv(pos, 'race_tracks_2022.csv')

In [118]:
race_tracks <- function(year_val){
    url <- sprintf('https://gpracingstats.com/seasons/%d-world-championship/', year_val)
    page <- read_html(url)
    pos <- page %>% html_node('.summary') %>% html_table(header = TRUE)
    pos$Circuit <- gsub(' ', '-', pos$Circuit)

    # List to be updated manually.
    pos$Circuit[pos$Circuit == 'Jeddah'] = 'Djeddah'
    pos$Circuit[pos$Circuit == 'Florida'] = 'Miami'
    pos$Circuit[pos$Circuit == 'Marina-Bay'] = 'Singapour'
    pos$Circuit[pos$Circuit == 'Monte-Carlo'] = 'Monaco'
    pos$Circuit[pos$Circuit == 'Catalunya'] = 'Barcelone'
    pos$Circuit[pos$Circuit == 'Baku'] = 'Bakou'
    pos$Circuit[pos$Circuit == 'Sepang'] = 'Kuala-Lumpur'
    pos$Circuit[pos$Circuit == 'Casablanca'] = 'Ain-Diab'
    pos$Circuit[pos$Circuit == 'Mosport'] = 'Mosport-Park'
    pos$Circuit[pos$Circuit == 'Buddh'] = 'New-Delhi'
    pos$Circuit[pos$Circuit == 'Jerez'] = 'Jerez-de-la-Frontera'

    
    pos <- pos %>% mutate('lat' = sapply(pos$Circuit, get_lat), 
                          'long' = sapply(pos$Circuit, get_long))

    write_csv(pos, sprintf('race_tracks_%d.csv', year_val))
}

In [129]:
years <- 2023

In [130]:
sapply(X = years,FUN = race_tracks)

No.,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22"
Grand Prix,"Bahrain , Saudi Arabia, Australia , Azerbaijan , Miami , Monaco , Spain , Canada , Austria , Britain , Hungary , Belgium , Netherlands , Italy , Singapore , Japan , Qatar , USA , Mexico City , Sao Paulo , Las Vegas , Abu Dhabi"
Circuit,"Sakhir , Djeddah , Melbourne , Bakou , Miami , Monaco , Barcelone , Montreal , Spielberg , Silverstone , Hungaroring , Spa-Francorchamps, Zandvoort , Monza , Singapour , Suzuka , Losail , Austin , Mexico-City , Interlagos , Las-Vegas , Yas-Marina"
Date,"05 March , 19 March , 02 April , 30 April , 07 May , 28 May , 04 June , 18 June , 02 July , 09 July , 23 July , 30 July , 27 August , 03 September, 17 September, 24 September, 08 October , 22 October , 29 October , 05 November , 18 November , 26 November"
lat,"26.030390, 21.638094, -37.849140, 40.371259, 25.957740, 43.738095, 41.568277, 45.505890, 47.223586, 52.071590, 47.581758, 50.438628, 52.387891, 45.618448, 1.287535, 34.843156, 25.490438, 30.133637, 19.400257, -23.703293, 36.117030, 24.471721"
long,"50.513543, 39.099443, 144.969873, 49.847693, -80.238678, 7.426050, 2.258206, -73.524110, 14.755540, -1.016150, 19.250748, 5.966606, 4.543061, 9.287676, 103.855477, 136.526698, 51.453857, -97.634573, -99.090279, -46.696961, -115.164946, 54.601836"


In [85]:
tracks

#,Grand Prix,Circuit,Date,lat,long
<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Bahrain,Sakhir,28 March,26.03039,50.513543
2,Emilia Romagna,Imola,18 April,44.33986,11.713329
3,Portugal,Portimao,02 May,37.22888,-8.627678
4,Spain,Barcelone,09 May,41.56828,2.258206
5,Monaco,Monaco,23 May,43.73809,7.426050
6,Azerbaijan,Bakou,06 June,40.37126,49.847693
7,France,Le-Castellet,20 June,43.25113,5.789497
8,Styria,Spielberg,27 June,47.22359,14.755540
9,Austria,Spielberg,04 July,47.22359,14.755540


In [117]:
write_csv(tracks, sprintf('race_tracks_%d.csv', years))

In [72]:
get_long('Kyalami')

[1] 28.06763

In [75]:
gplink <- 'https://gpracingstats.com/circuits/'

In [76]:
page <- read_html(gplink)

In [77]:
gptable <- page %>% html_nodes('.summary') %>% html_table(header = TRUE)

In [78]:
gptable

Circuit,Year(s) held,Races held
<chr>,<chr>,<int>
(AUS) Adelaide (Adelaide Street Circuit),1985-95,11
(JPN) Aida (Okayama International Circuit),1994-95,2
(GBR) Aintree (Aintree Motor Racing Circuit),"1955, 1957, 1959, 1961-62",5
(SWE) Anderstorp (Anderstorp Raceway),1973-78,6
(USA) Austin (Circuit of the Americas (COTA)),"2012-19, 2021-22",10
(DEU) AVUS (Automobil Verkehrs und Ubungsstrasse),1959,1
(AZE) Baku (Baku City Circuit),"2016-19, 2021-23",7
(GBR) Brands Hatch,"1964, 1966, 1968, 1970, 1972, 1974, 1976, 1978, 1980, 1982-86",14
(CHE) Bremgarten,1950-54,5


In [80]:
gptable <- data.frame(gptable)

In [81]:
write_csv(gptable, 'gptracks.csv')

In [97]:
tracks <- race_tracks(1986)

In [98]:
tracks

#,Grand Prix,Circuit,Date
<int>,<chr>,<chr>,<chr>
1,Brazil,Jacarepagua,23 March
2,Spain,Jerez,13 April
3,San Marino,Imola,27 April
4,Monaco,Monaco,11 May
5,Belgium,Spa-Francorchamps,25 May
6,Canada,Montreal,15 June
7,Detroit,Detroit,22 June
8,France,Le-Castellet,06 July
9,Britain,Brands-Hatch,13 July


In [100]:
get_lat('Jerez')

ERROR: Error in open.connection(x, "rb"): HTTP error 404.


In [116]:
tracks

#,Grand Prix,Circuit,Date,lat,long
<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Brazil,Jacarepagua,23 March,-22.97799,-43.3958898
2,Spain,Jerez-de-la-Frontera,13 April,36.70754,-6.0326723
3,San Marino,Imola,27 April,44.33986,11.7133288
4,Monaco,Monaco,11 May,43.73809,7.4260503
5,Belgium,Spa-Francorchamps,25 May,50.43863,5.9666064
6,Canada,Montreal,15 June,45.50589,-73.5241100
7,Detroit,Detroit,22 June,42.32801,-83.0405242
8,France,Le-Castellet,06 July,43.25113,5.7894971
9,Britain,Brands-Hatch,13 July,51.35554,0.2629122


In [52]:
url <- 'https://gpracingstats.com/circuits/adelaide/'

In [53]:
page <- read_html(url)

In [55]:
page %>% class()

[1] "xml_document" "xml_node"

In [4]:
library(httr)

In [5]:
page <- GET(url)

In [6]:
page

Response [https://www.statsf1.com/en/1950.aspx]
  Date: 2023-10-12 04:33
  Status: 200
  Content-Type: <unknown>
<EMPTY BODY>
NULL

In [3]:
page <- read_html(url)

ERROR: Error in read_xml.raw(raw, encoding = encoding, base_url = base_url, as_html = as_html, : Failed to parse text


In [205]:
library(httr)
ua <- "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"

In [207]:
page <- GET(url, user_agent(ua))

In [211]:
page

Response [https://statsf1.com/errors/500.jpg]
  Date: 2023-10-12 03:53
  Status: 200
  Content-Type: image/jpeg
  Size: 100 kB
<BINARY BODY>
NULL

In [209]:
circuits <- page %>% html_nodes('.pays') %>% html_text()

ERROR: Error in UseMethod("xml_find_all"): no applicable method for 'xml_find_all' applied to an object of class "response"


In [195]:
links <- page %>% html_nodes('.gp') %>% html_node('.flag') %>% html_node('a') %>% html_attr('href')

In [196]:
links

character(0)

In [197]:
circuits

character(0)

In [165]:
text <- "1. Britain"
result <- gsub("\\d+\\. ", "", text)
cat(result)

Britain

In [183]:
final <- data.frame(Circuits = gsub("\\d+\\. ", "", circuits),
                   Links = links)

In [184]:
final

Circuits,Links
<chr>,<chr>
Britain,/en/1950/grande-bretagne.aspx
Monaco,/en/1950/monaco.aspx
Indianapolis,/en/1950/indianapolis.aspx
Switzerland,/en/1950/suisse.aspx
Belgium,/en/1950/belgique.aspx
France,/en/1950/france.aspx
Italy,/en/1950/italie.aspx


In [2]:
race_circuits <- function(year){
    url <- sprintf('https://www.statsf1.com/en/%d.aspx', year)
    page <- read_html(url)
    circuits <- page %>% html_nodes('.pays') %>% html_text()
    links <- page %>% html_nodes('.gp') %>% html_node('.flag') %>% html_node('a') %>% html_attr('href')
    final <- data.frame(Circuits = gsub("\\d+\\. ", "", circuits),
                   Links = links)
    return(final)
}

In [3]:
race_circuits(1950)

ERROR: Error in read_xml.raw(raw, encoding = encoding, base_url = base_url, as_html = as_html, : Failed to parse text


In [12]:
user_agent("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36")

<request>
Options:
* useragent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36

In [7]:
require(httr)

cookies = c(
  `hasConsent` = "true",
  `IsMobile` = "false",
  `ASP.NET_SessionId` = "ya13brla3yavoidztkeng2ds"
)

headers = c(
  `Accept` = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
  `Accept-Language` = "en-US,en;q=0.9",
  `Cache-Control` = "no-cache",
  `Connection` = "keep-alive",
  `Pragma` = "no-cache",
  `Sec-Fetch-Dest` = "document",
  `Sec-Fetch-Mode` = "navigate",
  `Sec-Fetch-Site` = "none",
  `Sec-Fetch-User` = "?1",
  `Upgrade-Insecure-Requests` = "1",
  `User-Agent` = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
  `sec-ch-ua` = '"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
  `sec-ch-ua-mobile` = "?0",
  `sec-ch-ua-platform` = '"Windows"'
)

res <- httr::GET(url = "https://www.statsf1.com/en/1950.aspx", httr::add_headers(.headers=headers), httr::set_cookies(.cookies = cookies))

In [25]:
set_config(config = add_headers(.headers=headers), override = TRUE)

In [26]:
set_config(config = set_cookies(.cookies = cookies), override = TRUE)

In [18]:
add_headers(.headers=headers)

<request>
Headers:
* Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
* Accept-Language: en-US,en;q=0.9
* Cache-Control: no-cache
* Connection: keep-alive
* Pragma: no-cache
* Sec-Fetch-Dest: document
* Sec-Fetch-Mode: navigate
* Sec-Fetch-Site: none
* Sec-Fetch-User: ?1
* Upgrade-Insecure-Requests: 1
* User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36
* sec-ch-ua: "Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"
* sec-ch-ua-mobile: ?0
* sec-ch-ua-platform: "Windows"

In [27]:
url <- 'https://www.statsf1.com/en/1950.aspx'

In [28]:
url

[1] "https://www.statsf1.com/en/1950.aspx"

In [34]:
page <- read.xml(url)

ERROR: Error in read.xml(url): could not find function "read.xml"


In [9]:
res %>% class()

[1] "response"

In [11]:
rawToChar(res$content) %>% html

ERROR: Error in UseMethod("xml_find_all"): no applicable method for 'xml_find_all' applied to an object of class "character"


In [56]:
install.packages('polite')

also installing the dependencies 'credentials', 'zip', 'gitcreds', 'httr2', 'ini', 'spiderbar', 'desc', 'gert', 'gh', 'rprojroot', 'whisker', 'ratelimitr', 'robotstxt', 'usethis'





  There are binary versions available but the source versions are later:
            binary source needs_compilation
credentials  1.3.2  2.0.1             FALSE
httr2        0.2.2  0.2.3             FALSE
gert         1.9.2  2.0.0              TRUE
usethis      2.1.6  2.2.2             FALSE
polite       0.1.2  0.1.3             FALSE

  Binaries will be installed
package 'zip' successfully unpacked and MD5 sums checked
package 'gitcreds' successfully unpacked and MD5 sums checked
package 'ini' successfully unpacked and MD5 sums checked
package 'spiderbar' successfully unpacked and MD5 sums checked
package 'desc' successfully unpacked and MD5 sums checked
package 'gert' successfully unpacked and MD5 sums checked
package 'gh' successfully unpacked and MD5 sums checked
package 'rprojroot' successfully unpacked and MD5 sums checked
package 'whisker' successfully unpacked and MD5 sums checked
package 'ratelimitr' successfully unpacked and MD5 sums checked
package 'robotstxt' successfully 

installing the source packages 'credentials', 'httr2', 'usethis', 'polite'




In [3]:
library(polite)

In [60]:
session <- bow("https://www.statsf1.com/en/circuits.aspx", force = TRUE)
result <- scrape(session, query=list(t="semi-soft", per_page=100))

In [61]:
result

{html_document}
<html id="ctl00_html" lang="en">
[1] <head>\n<!-- Google Analytics --><!-- Global site tag (gtag.js) - Google  ...
[2] <body>\r\n  <form name="aspnetForm" method="post" action="/en/circuits.as ...

In [1]:
race_circuits <- function(year){
    session <- bow(sprintf("https://www.statsf1.com/en/%d.aspx", year), force = TRUE)
    result <- scrape(session, query=list(t="semi-soft", per_page=100))
    circuits <- result %>% html_nodes('.pays') %>% html_text2()
    links <- result %>% html_nodes('.gp') %>% html_node('.flag') %>% html_node('a') %>% html_attr('href')
    circuits <- gsub("\\d+\\. ", "", circuits)
    links <- gsub("\\.aspx$", "", links)
    circuits <- gsub("\r| ", "", circuits)
    final <- data.frame(Circuits = circuits,
                   Links = links)
    return(final)
}

In [5]:
merged_df <- race_circuits(2022)

In [6]:
merged_df

Circuits,Links
<chr>,<chr>
Bahrain,/en/2022/bahrein
SaudiArabia,/en/2022/arabie-saoudite
Australia,/en/2022/australie
Emilia-Romagna,/en/2022/emilie-romagne
Miami,/en/2022/miami
Spain,/en/2022/espagne
Monaco,/en/2022/monaco
Azerbaijan,/en/2022/azerbaidjan
Canada,/en/2022/canada


In [107]:
id <- merged_df$Circuits[3]

In [108]:
id

[1] "Indianapolis"

In [90]:
gsub("\r| ", "", id)


[1] "Indianapolis"

In [112]:
url <- sprintf('https://www.statsf1.com/%s/tour-par-tour.aspx', merged_df$Links[merged_df$Circuits == id])

In [113]:
url

[1] "https://www.statsf1.com//en/1950/indianapolis/tour-par-tour.aspx"

In [77]:
merged_df

Circuits,Links
<chr>,<chr>
Britain,/en/1950/grande-bretagne
Monaco,/en/1950/monaco
Indianapolis,/en/1950/indianapolis
Switzerland,/en/1950/suisse
Belgium,/en/1950/belgique
France,/en/1950/france
Italy,/en/1950/italie


In [111]:
merged_df$Links[merged_df$Circuits == 'Britain']

[1] "/en/1950/grande-bretagne"

In [119]:
a <- " "

In [120]:
if (a != " "){
    print('Hello')
}